In [ ]:
import re
import time
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from config import urls_1140412


In [ ]:
# # 預售屋建案查詢(2025/4/11)
# taipei = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/590f1fad4b56f04e9deed87e7f32c914?q=VTJGc2RHVmtYMTlyT2c0dkp3Mm1naitFeFljVndWVGFja2dGQVl3dkFRWXVNR3YwZXFMN05iZUthQ3dYMEQ3M3p0dldNZVdmSFM0S3dQdWNKRnJ0WG5LMjRNVXh5Smo2UGtqNXg0K2VsQTcrMHBReENhcHdVQzBLY040Q05TQXFsQWZWNkJhK2lkNG5nNUo4YTkvRy95ZU1uNjVMVmFRSDAreEVtVks0WG5nZUxXbHl2bStlY2tDdU5mdTBoWjd5MXE2MGpOUkxVQnJYRUtzYWpydXkxQ1JxMG8vT1NLR2pYSnFsdnZkdHhiYjRBa0p2WklES3BBZlVBQWlCenRsaFBmTVdGRXRZUXlKb2VxVGM4TGFhdnE2blZhQUx0ZzVYWFBoUzZwUWlBWEthSWVaQVZ1RWlXSlpkUE4zYnVCWXo="
# new_taipei = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8e282853cc329e3a49bf8fa49b6568f?q=VTJGc2RHVmtYMStOUlRzQVNnZk53MXpQOGtCQ0Z3R1pTN1E2SFdRc0J2ZzFVUlN2eHZwNk1xbXMxU25rNDFpU1dpUm4yVDA4SmFxN3NJVTNOd0ZQNkJMWmlGOXlWcS9MRmlKT2V0ZW9kL0NnaHdPaWhZMWU5Q1p6SWw0alNaTlRCSDZXMlRIamNnMnV5MVBuaVY4TXdTME42WldBUXhFOFhrREd3Y3J3VVFKNTR5VnFvVXpZcTByencwWW91SmNvc3NSNlFRV05wQ09KdzFRem1rTUh1QXFiR0UrZk0vekgyWGhoOVp3RjczNlM0NmRVQUZtTEV2VlplODZmcFB1eHNXRGN2Sy92bWRzTnMzbk5BZGVHd0lRUjNIS0dWQms0Qzl2aitxNUdkN2l6ME5ndFlMRTlSY2VGWE9PVk1oNGI="
# taoyuan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/dfbf3a3dd66ee9285a4ed1145e75c9b6?q=VTJGc2RHVmtYMTk5b25UU2NDUXBNOXJWMCt2WWMxTE1nYmZxN0pURC8rZGdIN1RodUlML2dtRkdNbDhaVEIzcDluU3FLY2VhRmRCM1VCRmttb3hRbnJOWCs5QmVkK3VobGJTeU5VZStneGpaUFFYaHZkbkQ5c0NBTFhrUmZ3T1NkS0x2L09FSWQ1emR2aE5DRitDRE9KSW0wUE02OHNDdkZWTEJSUDFBVytIZDFYRzVMUVpGQzVYMnc0ckJyODB5VjlaOXNkR25yUGxtTlRhd0x5YzBVUC9YcWZQbXhjM1l1RHYyT1BCTDZVczR6ZGxSclcwUFZrdmhEeWhSWXNpTHpwT3RRalpZcWtYdU81b2FBVllDRDNTYkZyYTB5QWozeWlWSnJlZ2dyUG0vKzRlVnBwa284REJMR0N2OWcwQmc="
# hsinchu_city = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8aef2f6fd3421e0f0f4f36251ee59ba?q=VTJGc2RHVmtYMStuMytZL1pFKzg2TjRSb294TWpsdFhyR0Y0MTF2Vmo1bHdQL1ozMi9SMkVGOWtkZVFRY0ZsMlBFanp3MG9jOFpQTWdJUVRTVDREQXE4SEhRUGJMRVdLemdzYTNZZlV1MzV5MS9rdGVDWStOenRHMDJzL3hjb3FkM2pVTitBMWpYOUl1OWVvUHAzVGVnQXpIbVhVUFI2REQ4SGJEUXZVTGJuTVVhdVNDNWg5dDJNdXk4K2l6a0d4QjBySWdoMVdzMUdQZzVqa1NxcWtnTEU2OEJ5N2JENGxqbGJnZHEwNzA4VW93VEdmVnBXTzArOTI5dEw0TmVyR0laYkxrVy9TTVhBZDdmNjg2a09KVHhRUTBkbjZ5ZjBRelZxQXphQldyVHZueEJZM3ZiMGI2eHhuY245N0d0Nng="
# hsinchu_county = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/e87ca53ad3a268cd8f560e13de4021b0?q=VTJGc2RHVmtYMTg1WXI4ak44SGdXaTBOdHdYdHY0cEJRbVBkbkJGbFN1c21meHhlMHpuNjQyV3FrTGEzR0hNdXF2cC9Sa0thMTBudmpUdzBSNktnM1ZkWmg3VlhidFZ0UldXUElPdWl0UWpWOGZBaWRTdWt6U0FORnZLTHAyeS9FUkVnRjF0K0JjeXZXNThVL3FNNndMd1lyZ0YyaDRVdFE4d05SOHpCZjFMeE8wcGM0MWhLa2NMOGlpejdqaFM3M0szV2ViQXIzN2FBZkh1NzhkVmJzWktXeVFOWEVRQ0J6MmxhN25nK3k3UzhvQUpTRFNxWFBwV3hPOTJ4ZXFzYUVQM0o5dURuZHNtdWF3UUNacy9rZ1VMV0NoeWJ6RkNoTmJ1d2JjSCtlSlR4eEpBZnY4ekdRL0RqaUN3NVo3Zlc="
# taichung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0837c4242a3659d6442b9a560a6ff252?q=VTJGc2RHVmtYMTlaemduSTBxbXFwRVFIQ2Z4RkRpNGppR29uZGJxdU1EOHhQVWpGTEY4Y3NGclkrYy9CMmtRa0NzM0hMczdGTDhRWFJmemMyeFR0RGpIcFdYZzRhR0FxdW04MFZPUWtDVndtVVlZMmp4ZVNjS3k5TWFMQjlpc05wZFF5cXQzUUtycTQwOXZFTE90TG5FOFJCVUZmVGZxMlh2SkQ5dzRNQlhOL1FIc0wxUHFGamNOakFHNTdrSFlKaTl2VjA3Sk1LRU5KS3JIbU1VeFZnSU9hc2gzeVBsRmlFNWpjT1YzRzNSUXdrclpOdkhlTU03Ymgvd3lxS05HMzExWk9ueklvL2tmSWxSZm81R0FweDRnZk9GQXYzeXM5amJWK1FmQzVZekJvWUVpak9GSlRhZHBNZ0YvYnVXREc="
# tainan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0e642025931344a4ec851cb0948e81d3?q=VTJGc2RHVmtYMTlPeTVLTUdDRzRrMlk2SkQ3K3VxVUtvWlZMaUgwMEFQWU41U0Flekx2NWJ2LzhidktTV3YvaWJtVzFkb0I5dnpWQ2piRU0xQ2VZcnlLMHJtWmNDTlc0UmRhc2JNQU9ud2ZLWW1kZklVd0prbmdSQVZnVjM0RS9wM1dQYnhYa3E0MzE3dnU4VmhiMFliWTB6ZHR5d1RWTVBQcjIyaDlkaUhYb3ZCRkpDRmdDTjlWejh1YnFGRktTTmtUSEZYZytYcWI2TVJ6L0lQdHV1VElzbmJ1cHFtUTltZjVvN2VpRm54YzdKb3NLaytFelRhTUJRdmE4OHBvZXpXVGpiY1A3bmNkQWNOU1pBMU9CVmFCbFA1SXV3V0x0YXNmT0dvTTBubllpQkNqbjBqb09kL214eGZoVm9PYnU="
# kaohsiung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c3ebdb0abff21c7c1df8e759fae523d4?q=VTJGc2RHVmtYMS9Td3ZmZ3BnOW02UGtjWTdCekdPbVpUUC9SUmN1S3N4Nks0YkpaVDhjOFdNUWNQNm8wWjlDL3o1YWhEZ1V3ZzkycWc3ZldHQ091WEN0bWV5Q2Fhc0FJL1NwVDhsV3FCckRrT3grL201VkkxZnVDTDJnUWJYcXVBbFJHOWo4dHoyK0dGQzNQaHN0SHo4d1J5bHJGcUFMSFRVbGhYSFByaUN4b0pEMnVmTDlqaTB4dGV6L0M5c2ZTa0pBcFl4SVBmMkcyL2U3emI3Q1FsMzFsMzYyRGJ2VTFZT1Y3UzVtK2tFVk90SlR5cHNPYVAxMFlXL29sK21xZmYyM3pOd1dUeXpQYkU1S3JHY0hLcytlMmsyMlB4SU5RR05FbDRDaFRZNDFxZHJrL3lJU3A4RmdoZmFyS0RXdHg="
# keelung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/3fcfceee4773076e77eb4f7acd71e48c?q=VTJGc2RHVmtYMSsrQVQ0ZWJyb0djbW0xRG5DZ0dEQUMwUXJ5YXpBVXdyL2lzTld4M0sxR1hRdHkvTjIvd3Y5UTN6OXcrbTZPYnd5SUhLQVk3eDRmRVRaajFNSmd6R3MrdnJ4SjJ6K0s2SWphYlZ3OFpLQUtvKzBNZVc0empVZC9qRUgxUWdsTFVHQmFVRzk4QzRXd1JiM0J5YVA5aUF2alBaV0VPQmxLTk5haEdyZDJxalNaaDRJMGJOTUc0SWFoUmN4aXR0b3ZZUHkvMkZtVnlySERNOEROcXBiOW1KN0N2bGozTi9JUiszWTd4VHFwNFJ1eWpxejR2U1pTZGNJb29NWlVqSnc4elBoaTR4b2JhUnEyY1V6b2FMOFRvU2FhZDFBbXJ1M1BFZVdUZGdDRXlNVWdrOTIwOGxDV1JsK28="
# chiayi_city = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/6c0818790c0f1b5a50fcb1d2afbd5472?q=VTJGc2RHVmtYMStQeHorMU5FZmFSWGszdnp2V015aEJsUm9TeUdvSGoyVUMvQ1ZCdlNsUlRyM2JiM29ab0p5UkJYcFhWTjk5WEZQZE45dmcwVE1MMlFEWVdBdUthOExlUHpNdE5BL1NBbzdidGZHOWRDaVV0VlNhY0tURGVWb3JYSDZLYTNtMjJ0WDlZKytVcDA2RDNMajErN2xvYVo4TDhZcW5naDMzTWhaSDdwVkhXMFFETWNjRldKcnFOYzZnSVpZMTc1cndrU2tscERBUmpJZnd1ci9pSVRhZzBxY3R1bHZ0TWdBRW1pUTRjYlgxanpoaXBxOXUxNWh6QXQzQjhTVXFoc2pVL1czTzFPT3RKTkVMRFpsc0dpK1VLenRoQnpsMjRxbWlSWTRtSGY4WStyYUpoa3ZQRXBCR3g3OGI="
# yilan = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/fc0fe973a7b4e0b3f2f1f71d97ee8488?q=VTJGc2RHVmtYMS9UUWlaYTRwOXVNUEp3MjhsTmI3T1BBZmxKSjRPSGhTalpUUEF5YnFXVUtPY3RWY2VvMldrWHd0c29zc2JIeFhWaGZkZE1FOUI2THVTZWFlM2RDVVpremtuQW5uVEJSS1VmM0NJakpHblo3RGNjeU1TVXA3WjhXUVpCYnRmT1JlTGJ6OUhqbEZ5aE1SN3VaNmNLOGVnbDRBTG42L3lIWlJ2MEtZTnBVYVJkS2d0cWZtU2F1RFl6Nm5qWjlvcTNpS1VBQVlORllMYjZzcU1TNWJ4aHZLVnR5SHdDc1QwdXNtK2F3ajZVRUtKSCtxeHhXUzZiakFqY0h4blFqSDlIR01FY3pWREFHcnhudVJtbi9UZWxncHAzQ0NBNGpoMUJEME02dWNUVmRHYUVZNUkrZ3lKaE5FZXE="  
# miaoli = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/ea88499f21a09c39cfb32ba66852f9bd?q=VTJGc2RHVmtYMTk4OEh3N1hIc1N4K2RrdS9RZzFZZUFqQ2FpL05rTnJQQ3p3STU2VnQ3b0ZscEhWbkx5dmRRLzkwK0lyekhoM1l1TFo3RGN0NWRZWUo4S2NWWDVveCttSGVWS09mZFpxdnViSDFwc0VyS1B6a1c2WlNmNk1sYmJtd1Fhd2lUbWlDYzJIWUp0QjIwT2RqeFFUOEJUWHBmQVpJa0hRbzh0dWErSURabTBqVEE5akwxRzJvN0F3dzY1R3RVMEMwMjlYYzBLUnZ0MVl2WkJNbTE5eFd0dmgveUhrWHBxZlJuZVByNmNJajlHeUtVSDVGeVA1N2w5QnB4dlhBck9NUWJQTmZMSEhwMmtzcWlJNk1hWGRSbGFPcEJBZHBiSjFudmZ1TlFRS1RRTyt1c1MxdnEvUVg5aG9JcUg="
# changhua = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/4d038ed90563ca35f2333ce2493d401d?q=VTJGc2RHVmtYMTlWdCtoZTRaVEo4dWJpYXZmRFZWcWUwWFNQdTFkdDBvWWVucVlvK3VkMWpaOG4rK1Z4ZWRoSzNhTkY1di9CdEF6VW1NSmU1RUJnbFJEQS9PL0VrY2ZMc1hDdUwrZjYzQU5QaU4rSHVCRTdUNkY5dkRFQTRDS0hGVFNXMCtUOUdkbkV3STk3REUrRFg5RnBqR3BPNlN1c1JKWUxrbCtzT1F2Vmg0YzkyYjRMZTJQbkFxQjJDNFNlTkRUcjc2NnNRVGNQUVRXSk5ueGZrOG9Td3RpOFgwOHJpb2FURTNQNVJIV1ZyZHhiV1JlYkZ1eU9MV0NZVy9TbisvNXRWNTBBdUVEdUgyR1ZFaHdReVVlY056cDJ4ZGdmOXdwdDlzZitjbWdYS0U4Z3daUEpob3dWTzFYM0RiazU=" 
# nantou = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/1ec0e19e86896b0e11668aa4832e73f9?q=VTJGc2RHVmtYMS9CWjRnMzZYKzQydk5VSjFyaFdTc2loQjYzREpKTEZTTHBsRXZaOG9RT2RIY0lRd2V3YVNZL1lJTk1Ib3E2cVhRNDFGQzhqRTVONXE1SUFoYkxCbmpHZGZlSGRFZ0hkamQwNFdLeExjSEFEU1dHYlE1TktGQVZoZWlNS3BBWC9Jb29JQ1oxNllYTjlQSHY3bXdqVUdJVVJzSmV1Y1RPajFwWkZmV2M2TVNRUHFWV1lBMjdrWkw3ZXJzbFQ1SnN6L1dWZ1BpaTY0NGU5UlcwOThKQ1h2aHNSMFB3QjNNaG1waXN4UElNZkRZU3BubFN6T1gzUUxmR1FHbVFhVUdWYlJvUUNrUHhmZXA1R1lldnU1YWplaGFKTWxIeWkvQUxDNUF4RDdPU2NOZHVoQWIya0wwRzE0R2o="
# yunlin = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5cb21bf504adf65943a3e28b653d55f9?q=VTJGc2RHVmtYMS85UFBkRk16RWxFdGltLytiaG1vZ2o2aGVnZHZHR0dZWEU4eWhaaXpqMlB3UGVuTDArYTVPd25PQ2cvZUgwRGVzWDN3dFpDQjM0VnR4U29aSW5GZWluVks2cjZaRUFkOUM2QWJWK3ZUOFU3VGpJUHZiNWRVRXc3Y2h4dnFsckJBVndpMEE3VFZ3Y1A2bGc5TUc5anVNRHNEeGc3bmF0L1kyREJhZXpRMzZUeDJnUExoZ3NuaXZsSlV4T1RBbHI3Uk1uZlJmKzFxdjFMenI3ekRnWWRSREVnWjh4YU9ybWV1NXNCM3QrUStZT3hzSjJrUjh0R1U5SkdzQlhVR1hZclY2dXhwVDd6TEU2SWsvT1V4ZTlMK1EzL28vS1BVTldhOVh3MTc2RUZyRWpiL25NNkZUUFBIVnQ="
# chiayi_county = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/93b64316954079655a1f3f6bbf4a58a8?q=VTJGc2RHVmtYMSt3eW41SmE3ME1LUVhyWTB1NXhNYlhvYlFFOHA3VHkwQkZqWThidWpkSjJjU2k4UkkvWkpMbFNVb3NoZTZ2N1duVnFIOWxuQ0FBUnlnUkVYOGN1Rk5xdWNlZEd6Q0xFZVdORjVPR2NBMUZMbzVxczdBQVA5S2pJR0lLZGlkdDJ0SzltOG5yVWNLNjlaRkxvVEVId2h0SHRCN21LY0craWFZQ2M3ZEk2OGhTSUFZNEV3RFpQcHR0NzYxalMvSjNkcFhJOWMrbTNOZzAyOG4wbkJlZHZ6UWVJcXArcUt0b3poL25DK1VtbmhXQU9JdmNVeHR4VHBqaHBsa3pia2xXY3U3anlJWTlYS3R5SndtSHp2TzgxV09nMi8wN2FoRVdOU2tubjF1TEZnaXBQSXFldHZBbURYYSs="
# pingtung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c6e86f5f1d44360781c3e51fc7a5fb22?q=VTJGc2RHVmtYMS9NVXM0bUhyQlFFTzRLbjVzQStCQzcyR3l0d1N1cGk1U29jSktDd1BZSTBYUUFZckhHa1VYbEhXT29iUHNmVkJUeGpFdnRlOUhIazU0OWkyRXhBTXV6SHd0L1Fjc2gwV1laQ0MySGg0OEJnWU5Zc280YW1LL1lPWHA4OWFUekNLOXp6YzlUUzNOL3VycittL3lJUVA4VEw0cXdxNFJxdFE1b1NPTytOUmJhY0VPdnBUcStHSkNzemFHN0V0QnM1bHFiT1l5enp6a2lKeVhFbGw1ZWdPWjhoZDlwa1ZVam00VUxxOHR6V1VlUWRLajY0cE1Xc1ZZOGVIYTNWOG1wTllWV0lsYVlTaXRyeU9lUER0a0R1WG9UaHlvUzd3R0puVUpWK2hFaU9NQ05lVU1FSlVUeCsxOXM="
# taitung = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5f00ba2284ad4f8b31db0e6fe95c96a7?q=VTJGc2RHVmtYMS9teFNDUmR6dDRQNTdNOXJDVkV0TURkemVrVUVBYWRuTWFYODF3T2dLc1o4K091ZDZNUzBLZ3QrUjdUWG9IVGNRMm96ZDI4VzFZU1hrQVJOc3RMRGE3NnREeFl1VW5lOXZFQUVnNjBkM2RwbWtZYlBHaURSMzV3QVhETDFhRkFMUlV3R1Q2ZExlalp4eEcrWVNPWFlhQlFxcUNhZG5JZFdnNUdVL2szR3BqeCtzQmZ2anpqc1o3VUM2T0tYcUU4K1FRSElhZmx0dXc2cGp1bHk2d2haQU90U1hkbWFZQStkZUFrSE5ZRXFIbXJnMVlZWm1XTzJETElLWFJYVDBpdFN3YWNsVGhSMG9EaU8rd0RubEorcXVYVytWZ3pvYWpSSmY4bUdOTlA2VzMvWUtiL2hua0ZVM1c="
# hualien = "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/107f2b57838510c356e3b5d9b660e606?q=VTJGc2RHVmtYMTkyajZiTzAraEtITjl1SFdtK2ZSeTdwU3cwckthalM5YkVCLzRRcHJLd09kWWNzd2RIYmxzQWFzTW9GVUFoZzdzTXZtS3dXNktMZjlITG9BeCt2MWxmdHFLbkNweUpuS3IwWjZKYU0xSTRHZ21nWWQrUFpmckhWK3ZNc1NUV3lhRkFsb1dqcUx5QVFoaEF0SURNVXpPY0hJOVMrMTBmcDNoYU40bDBVNEVuMXd3U1duN1VqSmZGT0hsWm9yZzErdEc1L3NWd0FiU2lmRngvamVSb3ZCazViYzlOcHZvYmxydnlCWTdtd3hwYnZLRUVuN1dsMDd3RnI0YWVuRkxhckZHVXN2bUErUlYwSHpJblk5YVZuaC9JWjJULzRDbEVtMlUwc3BXRU0zUE1OcFVTNWptdWFRVmU="



In [ ]:
res = requests.get("https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/4952081bde320989e6707a12115e58c5?q=VTJGc2RHVmtYMTlaUTFoSHJDUVdRRE5ZbTdWaFhkYWdLU0ZlT1pEZnIveERSTVpSM2k1SmYweEN6OGtMc3oxMHZvYlVjNlFBcUFoWWVHK0xGbCttR0wxVVpvTENhWEMrTjZTVkFtQjhXTGtBV2ZwMDBlN3dlajdqME4rQVIweHNhZ09DMGFQdjZBOFRkdW5yMGRzMXB2RExQc3BxTGkxRTZVUDZPZWQ1T21QOGdnSFpaVHlsdkdhQWN6ckNSTXh6enoyY25BdGdTbW43Rnp5cG1jZGFLTVU5MTc3K3dzWmtRYWFFVkFLNk9iTFZBN29pUjN1ZHIzcnZmaE5jYUxQTkFidVBpZWExTWd4enVzVkFBR3F6UmRmVWZJcitjdDdySFBpSTdNSGxSaUxOamd1SUl2MkUrYkhWNTdiaFBNeVpLajFJZG1sSExCbExxSDNuZ3FvbUVaMDVRMlNHSDcvNVVoT1ovU3FQbEU4R2o5RXpsMHphMGJNRk9YQjhyeWh3dFh3bUZNVnBVQW5YN2M3Vm1NUTFoSy82eUM3YmlMZzRvbVpyN01oZlNLYU9mbkF3Z1Z6OXQ2SGMybTE1MXo5Y3ZKbEp0ekFMMzM0M3N2NGx1NVh5YUlOeVQ5OFNhc01JZW5mY2dkUzVkWWhkcmoxdUU3SjZyTTV4bE5ac1V5Slo0ejlpVFNkMTdLYThRdS9nTVlFNDZiVUVXRlQ5ZElpNFl0bisrYTB0MzhuTWlOUDV2azg4bDNFa0Z5WUd1L3psRFJnUEJ1SFc0aUp1YWNSaU1JMjIxU3pST095K292czN1elZackYwTlE2Y3A3c3pPdlNYT2J5dXJqYW5sU0g5ZHh0NzdmWktVSTRXTUFaK0YzQlptQUxtc3FPckVKaGNxVWM1dGpFYUtHbzVVaFJBQkRRR00rNzIvb3Z3YkZhZHJ3ak5MaTJrM3ZDNGp3TGFOb1I4N3hSYlRCQ1BwZHowT0dzYmYzaEVPMzlNPQ==")
data = data = res.json()

In [ ]:
len(data)

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df['e']

In [ ]:
urls = {
    "臺北市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/590f1fad4b56f04e9deed87e7f32c914?q=VTJGc2RHVmtYMTlyT2c0dkp3Mm1naitFeFljVndWVGFja2dGQVl3dkFRWXVNR3YwZXFMN05iZUthQ3dYMEQ3M3p0dldNZVdmSFM0S3dQdWNKRnJ0WG5LMjRNVXh5Smo2UGtqNXg0K2VsQTcrMHBReENhcHdVQzBLY040Q05TQXFsQWZWNkJhK2lkNG5nNUo4YTkvRy95ZU1uNjVMVmFRSDAreEVtVks0WG5nZUxXbHl2bStlY2tDdU5mdTBoWjd5MXE2MGpOUkxVQnJYRUtzYWpydXkxQ1JxMG8vT1NLR2pYSnFsdnZkdHhiYjRBa0p2WklES3BBZlVBQWlCenRsaFBmTVdGRXRZUXlKb2VxVGM4TGFhdnE2blZhQUx0ZzVYWFBoUzZwUWlBWEthSWVaQVZ1RWlXSlpkUE4zYnVCWXo=",
    # "新北市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8e282853cc329e3a49bf8fa49b6568f?q=VTJGc2RHVmtYMStOUlRzQVNnZk53MXpQOGtCQ0Z3R1pTN1E2SFdRc0J2ZzFVUlN2eHZwNk1xbXMxU25rNDFpU1dpUm4yVDA4SmFxN3NJVTNOd0ZQNkJMWmlGOXlWcS9MRmlKT2V0ZW9kL0NnaHdPaWhZMWU5Q1p6SWw0alNaTlRCSDZXMlRIamNnMnV5MVBuaVY4TXdTME42WldBUXhFOFhrREd3Y3J3VVFKNTR5VnFvVXpZcTByencwWW91SmNvc3NSNlFRV05wQ09KdzFRem1rTUh1QXFiR0UrZk0vekgyWGhoOVp3RjczNlM0NmRVQUZtTEV2VlplODZmcFB1eHNXRGN2Sy92bWRzTnMzbk5BZGVHd0lRUjNIS0dWQms0Qzl2aitxNUdkN2l6ME5ndFlMRTlSY2VGWE9PVk1oNGI=",
    # "桃園市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/dfbf3a3dd66ee9285a4ed1145e75c9b6?q=VTJGc2RHVmtYMTk5b25UU2NDUXBNOXJWMCt2WWMxTE1nYmZxN0pURC8rZGdIN1RodUlML2dtRkdNbDhaVEIzcDluU3FLY2VhRmRCM1VCRmttb3hRbnJOWCs5QmVkK3VobGJTeU5VZStneGpaUFFYaHZkbkQ5c0NBTFhrUmZ3T1NkS0x2L09FSWQ1emR2aE5DRitDRE9KSW0wUE02OHNDdkZWTEJSUDFBVytIZDFYRzVMUVpGQzVYMnc0ckJyODB5VjlaOXNkR25yUGxtTlRhd0x5YzBVUC9YcWZQbXhjM1l1RHYyT1BCTDZVczR6ZGxSclcwUFZrdmhEeWhSWXNpTHpwT3RRalpZcWtYdU81b2FBVllDRDNTYkZyYTB5QWozeWlWSnJlZ2dyUG0vKzRlVnBwa284REJMR0N2OWcwQmc=",
    # "新竹市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/a8aef2f6fd3421e0f0f4f36251ee59ba?q=VTJGc2RHVmtYMStuMytZL1pFKzg2TjRSb294TWpsdFhyR0Y0MTF2Vmo1bHdQL1ozMi9SMkVGOWtkZVFRY0ZsMlBFanp3MG9jOFpQTWdJUVRTVDREQXE4SEhRUGJMRVdLemdzYTNZZlV1MzV5MS9rdGVDWStOenRHMDJzL3hjb3FkM2pVTitBMWpYOUl1OWVvUHAzVGVnQXpIbVhVUFI2REQ4SGJEUXZVTGJuTVVhdVNDNWg5dDJNdXk4K2l6a0d4QjBySWdoMVdzMUdQZzVqa1NxcWtnTEU2OEJ5N2JENGxqbGJnZHEwNzA4VW93VEdmVnBXTzArOTI5dEw0TmVyR0laYkxrVy9TTVhBZDdmNjg2a09KVHhRUTBkbjZ5ZjBRelZxQXphQldyVHZueEJZM3ZiMGI2eHhuY245N0d0Nng=",
    # "新竹縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/e87ca53ad3a268cd8f560e13de4021b0?q=VTJGc2RHVmtYMTg1WXI4ak44SGdXaTBOdHdYdHY0cEJRbVBkbkJGbFN1c21meHhlMHpuNjQyV3FrTGEzR0hNdXF2cC9Sa0thMTBudmpUdzBSNktnM1ZkWmg3VlhidFZ0UldXUElPdWl0UWpWOGZBaWRTdWt6U0FORnZLTHAyeS9FUkVnRjF0K0JjeXZXNThVL3FNNndMd1lyZ0YyaDRVdFE4d05SOHpCZjFMeE8wcGM0MWhLa2NMOGlpejdqaFM3M0szV2ViQXIzN2FBZkh1NzhkVmJzWktXeVFOWEVRQ0J6MmxhN25nK3k3UzhvQUpTRFNxWFBwV3hPOTJ4ZXFzYUVQM0o5dURuZHNtdWF3UUNacy9rZ1VMV0NoeWJ6RkNoTmJ1d2JjSCtlSlR4eEpBZnY4ekdRL0RqaUN3NVo3Zlc=",
    # "臺中市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0837c4242a3659d6442b9a560a6ff252?q=VTJGc2RHVmtYMTlaemduSTBxbXFwRVFIQ2Z4RkRpNGppR29uZGJxdU1EOHhQVWpGTEY4Y3NGclkrYy9CMmtRa0NzM0hMczdGTDhRWFJmemMyeFR0RGpIcFdYZzRhR0FxdW04MFZPUWtDVndtVVlZMmp4ZVNjS3k5TWFMQjlpc05wZFF5cXQzUUtycTQwOXZFTE90TG5FOFJCVUZmVGZxMlh2SkQ5dzRNQlhOL1FIc0wxUHFGamNOakFHNTdrSFlKaTl2VjA3Sk1LRU5KS3JIbU1VeFZnSU9hc2gzeVBsRmlFNWpjT1YzRzNSUXdrclpOdkhlTU03Ymgvd3lxS05HMzExWk9ueklvL2tmSWxSZm81R0FweDRnZk9GQXYzeXM5amJWK1FmQzVZekJvWUVpak9GSlRhZHBNZ0YvYnVXREc=",
    # "臺南市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/0e642025931344a4ec851cb0948e81d3?q=VTJGc2RHVmtYMTlPeTVLTUdDRzRrMlk2SkQ3K3VxVUtvWlZMaUgwMEFQWU41U0Flekx2NWJ2LzhidktTV3YvaWJtVzFkb0I5dnpWQ2piRU0xQ2VZcnlLMHJtWmNDTlc0UmRhc2JNQU9ud2ZLWW1kZklVd0prbmdSQVZnVjM0RS9wM1dQYnhYa3E0MzE3dnU4VmhiMFliWTB6ZHR5d1RWTVBQcjIyaDlkaUhYb3ZCRkpDRmdDTjlWejh1YnFGRktTTmtUSEZYZytYcWI2TVJ6L0lQdHV1VElzbmJ1cHFtUTltZjVvN2VpRm54YzdKb3NLaytFelRhTUJRdmE4OHBvZXpXVGpiY1A3bmNkQWNOU1pBMU9CVmFCbFA1SXV3V0x0YXNmT0dvTTBubllpQkNqbjBqb09kL214eGZoVm9PYnU=",
    # "高雄市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c3ebdb0abff21c7c1df8e759fae523d4?q=VTJGc2RHVmtYMS9Td3ZmZ3BnOW02UGtjWTdCekdPbVpUUC9SUmN1S3N4Nks0YkpaVDhjOFdNUWNQNm8wWjlDL3o1YWhEZ1V3ZzkycWc3ZldHQ091WEN0bWV5Q2Fhc0FJL1NwVDhsV3FCckRrT3grL201VkkxZnVDTDJnUWJYcXVBbFJHOWo4dHoyK0dGQzNQaHN0SHo4d1J5bHJGcUFMSFRVbGhYSFByaUN4b0pEMnVmTDlqaTB4dGV6L0M5c2ZTa0pBcFl4SVBmMkcyL2U3emI3Q1FsMzFsMzYyRGJ2VTFZT1Y3UzVtK2tFVk90SlR5cHNPYVAxMFlXL29sK21xZmYyM3pOd1dUeXpQYkU1S3JHY0hLcytlMmsyMlB4SU5RR05FbDRDaFRZNDFxZHJrL3lJU3A4RmdoZmFyS0RXdHg=",
    # "基隆市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/3fcfceee4773076e77eb4f7acd71e48c?q=VTJGc2RHVmtYMSsrQVQ0ZWJyb0djbW0xRG5DZ0dEQUMwUXJ5YXpBVXdyL2lzTld4M0sxR1hRdHkvTjIvd3Y5UTN6OXcrbTZPYnd5SUhLQVk3eDRmRVRaajFNSmd6R3MrdnJ4SjJ6K0s2SWphYlZ3OFpLQUtvKzBNZVc0empVZC9qRUgxUWdsTFVHQmFVRzk4QzRXd1JiM0J5YVA5aUF2alBaV0VPQmxLTk5haEdyZDJxalNaaDRJMGJOTUc0SWFoUmN4aXR0b3ZZUHkvMkZtVnlySERNOEROcXBiOW1KN0N2bGozTi9JUiszWTd4VHFwNFJ1eWpxejR2U1pTZGNJb29NWlVqSnc4elBoaTR4b2JhUnEyY1V6b2FMOFRvU2FhZDFBbXJ1M1BFZVdUZGdDRXlNVWdrOTIwOGxDV1JsK28=",
    # "嘉義市": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/6c0818790c0f1b5a50fcb1d2afbd5472?q=VTJGc2RHVmtYMStQeHorMU5FZmFSWGszdnp2V015aEJsUm9TeUdvSGoyVUMvQ1ZCdlNsUlRyM2JiM29ab0p5UkJYcFhWTjk5WEZQZE45dmcwVE1MMlFEWVdBdUthOExlUHpNdE5BL1NBbzdidGZHOWRDaVV0VlNhY0tURGVWb3JYSDZLYTNtMjJ0WDlZKytVcDA2RDNMajErN2xvYVo4TDhZcW5naDMzTWhaSDdwVkhXMFFETWNjRldKcnFOYzZnSVpZMTc1cndrU2tscERBUmpJZnd1ci9pSVRhZzBxY3R1bHZ0TWdBRW1pUTRjYlgxanpoaXBxOXUxNWh6QXQzQjhTVXFoc2pVL1czTzFPT3RKTkVMRFpsc0dpK1VLenRoQnpsMjRxbWlSWTRtSGY4WStyYUpoa3ZQRXBCR3g3OGI=",
    # "宜蘭縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/fc0fe973a7b4e0b3f2f1f71d97ee8488?q=VTJGc2RHVmtYMS9UUWlaYTRwOXVNUEp3MjhsTmI3T1BBZmxKSjRPSGhTalpUUEF5YnFXVUtPY3RWY2VvMldrWHd0c29zc2JIeFhWaGZkZE1FOUI2THVTZWFlM2RDVVpremtuQW5uVEJSS1VmM0NJakpHblo3RGNjeU1TVXA3WjhXUVpCYnRmT1JlTGJ6OUhqbEZ5aE1SN3VaNmNLOGVnbDRBTG42L3lIWlJ2MEtZTnBVYVJkS2d0cWZtU2F1RFl6Nm5qWjlvcTNpS1VBQVlORllMYjZzcU1TNWJ4aHZLVnR5SHdDc1QwdXNtK2F3ajZVRUtKSCtxeHhXUzZiakFqY0h4blFqSDlIR01FY3pWREFHcnhudVJtbi9UZWxncHAzQ0NBNGpoMUJEME02dWNUVmRHYUVZNUkrZ3lKaE5FZXE=",
    # "苗栗縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/ea88499f21a09c39cfb32ba66852f9bd?q=VTJGc2RHVmtYMTk4OEh3N1hIc1N4K2RrdS9RZzFZZUFqQ2FpL05rTnJQQ3p3STU2VnQ3b0ZscEhWbkx5dmRRLzkwK0lyekhoM1l1TFo3RGN0NWRZWUo4S2NWWDVveCttSGVWS09mZFpxdnViSDFwc0VyS1B6a1c2WlNmNk1sYmJtd1Fhd2lUbWlDYzJIWUp0QjIwT2RqeFFUOEJUWHBmQVpJa0hRbzh0dWErSURabTBqVEE5akwxRzJvN0F3dzY1R3RVMEMwMjlYYzBLUnZ0MVl2WkJNbTE5eFd0dmgveUhrWHBxZlJuZVByNmNJajlHeUtVSDVGeVA1N2w5QnB4dlhBck9NUWJQTmZMSEhwMmtzcWlJNk1hWGRSbGFPcEJBZHBiSjFudmZ1TlFRS1RRTyt1c1MxdnEvUVg5aG9JcUg=",
    # "彰化縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/4d038ed90563ca35f2333ce2493d401d?q=VTJGc2RHVmtYMTlWdCtoZTRaVEo4dWJpYXZmRFZWcWUwWFNQdTFkdDBvWWVucVlvK3VkMWpaOG4rK1Z4ZWRoSzNhTkY1di9CdEF6VW1NSmU1RUJnbFJEQS9PL0VrY2ZMc1hDdUwrZjYzQU5QaU4rSHVCRTdUNkY5dkRFQTRDS0hGVFNXMCtUOUdkbkV3STk3REUrRFg5RnBqR3BPNlN1c1JKWUxrbCtzT1F2Vmg0YzkyYjRMZTJQbkFxQjJDNFNlTkRUcjc2NnNRVGNQUVRXSk5ueGZrOG9Td3RpOFgwOHJpb2FURTNQNVJIV1ZyZHhiV1JlYkZ1eU9MV0NZVy9TbisvNXRWNTBBdUVEdUgyR1ZFaHdReVVlY056cDJ4ZGdmOXdwdDlzZitjbWdYS0U4Z3daUEpob3dWTzFYM0RiazU=",
    # "南投縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/341cd3107a94ed892fe54a88b0cf19bb?q=VTJGc2RHVmtYMStybGhvZjBld3J3WUVxTUF3bTlTM21uci9aUDdLYjhwa1hMditnL0UvYUc0dllDSTRIU2w5c21FZjIranVVblpONWdHSVY4Wi9HdGhjdm51NEhKT1k4eDZBT2N0RUkzVnk4azhlYmJmVjI3S0t3bUd5ZVRWNlZ0bVQ5bTd3aUNJWUVmdkp1NjlSSkdiSnNnQTAvVUVXNnFWdVFEcHdYWmVqWHRyUTMrNU5IcEl6RWVMV0dVUWI0aXFUc0ZXRG94YnFjWnIwZjhzRmtBUTZqUXFzS1VOSndKRTgxYWtrMFk5cEU1dUR0cTJDZ1hjVTlFUWxLQWczMURtRXY0WFR2cEMrK2RXNTZENGptWXZFUW1QVmxSVHVCdDh0SjkxRnFoUlo5KzJVWnRCbUc0TUxlMWpIWHhodVo=",
    # "雲林縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5cb21bf504adf65943a3e28b653d55f9?q=VTJGc2RHVmtYMS85UFBkRk16RWxFdGltLytiaG1vZ2o2aGVnZHZHR0dZWEU4eWhaaXpqMlB3UGVuTDArYTVPd25PQ2cvZUgwRGVzWDN3dFpDQjM0VnR4U29aSW5GZWluVks2cjZaRUFkOUM2QWJWK3ZUOFU3VGpJUHZiNWRVRXc3Y2h4dnFsckJBVndpMEE3VFZ3Y1A2bGc5TUc5anVNRHNEeGc3bmF0L1kyREJhZXpRMzZUeDJnUExoZ3NuaXZsSlV4T1RBbHI3Uk1uZlJmKzFxdjFMenI3ekRnWWRSREVnWjh4YU9ybWV1NXNCM3QrUStZT3hzSjJrUjh0R1U5SkdzQlhVR1hZclY2dXhwVDd6TEU2SWsvT1V4ZTlMK1EzL28vS1BVTldhOVh3MTc2RUZyRWpiL25NNkZUUFBIVnQ=",
    # "嘉義縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/93b64316954079655a1f3f6bbf4a58a8?q=VTJGc2RHVmtYMSt3eW41SmE3ME1LUVhyWTB1NXhNYlhvYlFFOHA3VHkwQkZqWThidWpkSjJjU2k4UkkvWkpMbFNVb3NoZTZ2N1duVnFIOWxuQ0FBUnlnUkVYOGN1Rk5xdWNlZEd6Q0xFZVdORjVPR2NBMUZMbzVxczdBQVA5S2pJR0lLZGlkdDJ0SzltOG5yVWNLNjlaRkxvVEVId2h0SHRCN21LY0craWFZQ2M3ZEk2OGhTSUFZNEV3RFpQcHR0NzYxalMvSjNkcFhJOWMrbTNOZzAyOG4wbkJlZHZ6UWVJcXArcUt0b3poL25DK1VtbmhXQU9JdmNVeHR4VHBqaHBsa3pia2xXY3U3anlJWTlYS3R5SndtSHp2TzgxV09nMi8wN2FoRVdOU2tubjF1TEZnaXBQSXFldHZBbURYYSs=",
    # "屏東縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/c6e86f5f1d44360781c3e51fc7a5fb22?q=VTJGc2RHVmtYMS9NVXM0bUhyQlFFTzRLbjVzQStCQzcyR3l0d1N1cGk1U29jSktDd1BZSTBYUUFZckhHa1VYbEhXT29iUHNmVkJUeGpFdnRlOUhIazU0OWkyRXhBTXV6SHd0L1Fjc2gwV1laQ0MySGg0OEJnWU5Zc280YW1LL1lPWHA4OWFUekNLOXp6YzlUUzNOL3VycittL3lJUVA4VEw0cXdxNFJxdFE1b1NPTytOUmJhY0VPdnBUcStHSkNzemFHN0V0QnM1bHFiT1l5enp6a2lKeVhFbGw1ZWdPWjhoZDlwa1ZVam00VUxxOHR6V1VlUWRLajY0cE1Xc1ZZOGVIYTNWOG1wTllWV0lsYVlTaXRyeU9lUER0a0R1WG9UaHlvUzd3R0puVUpWK2hFaU9NQ05lVU1FSlVUeCsxOXM=",
    # "臺東縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/5f00ba2284ad4f8b31db0e6fe95c96a7?q=VTJGc2RHVmtYMS9teFNDUmR6dDRQNTdNOXJDVkV0TURkemVrVUVBYWRuTWFYODF3T2dLc1o4K091ZDZNUzBLZ3QrUjdUWG9IVGNRMm96ZDI4VzFZU1hrQVJOc3RMRGE3NnREeFl1VW5lOXZFQUVnNjBkM2RwbWtZYlBHaURSMzV3QVhETDFhRkFMUlV3R1Q2ZExlalp4eEcrWVNPWFlhQlFxcUNhZG5JZFdnNUdVL2szR3BqeCtzQmZ2anpqc1o3VUM2T0tYcUU4K1FRSElhZmx0dXc2cGp1bHk2d2haQU90U1hkbWFZQStkZUFrSE5ZRXFIbXJnMVlZWm1XTzJETElLWFJYVDBpdFN3YWNsVGhSMG9EaU8rd0RubEorcXVYVytWZ3pvYWpSSmY4bUdOTlA2VzMvWUtiL2hua0ZVM1c=",
    # "花蓮縣": "https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/SaleData/107f2b57838510c356e3b5d9b660e606?q=VTJGc2RHVmtYMTkyajZiTzAraEtITjl1SFdtK2ZSeTdwU3cwckthalM5YkVCLzRRcHJLd09kWWNzd2RIYmxzQWFzTW9GVUFoZzdzTXZtS3dXNktMZjlITG9BeCt2MWxmdHFLbkNweUpuS3IwWjZKYU0xSTRHZ21nWWQrUFpmckhWK3ZNc1NUV3lhRkFsb1dqcUx5QVFoaEF0SURNVXpPY0hJOVMrMTBmcDNoYU40bDBVNEVuMXd3U1duN1VqSmZGT0hsWm9yZzErdEc1L3NWd0FiU2lmRngvamVSb3ZCazViYzlOcHZvYmxydnlCWTdtd3hwYnZLRUVuN1dsMDd3RnI0YWVuRkxhckZHVXN2bUErUlYwSHpJblk5YVZuaC9JWjJULzRDbEVtMlUwc3BXRU0zUE1OcFVTNWptdWFRVmU="
}

In [ ]:

# 以手動更新取得的urls，再利用 requests 取得於實價登錄網站取回 JSON 資料並回傳 DataFrame
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 若有錯誤狀況，會引發例外
        data = response.json()
        return pd.DataFrame(data)
    except Exception as e:
        print(f"取得資料時發生錯誤：{e}")
        return pd.DataFrame()  # 回傳空的 DataFrame

# 建立一個空的列表存放各區的 DataFrame
df_list = []

# 迴圈走訪所有 URL，並新增一個表示地區的欄位
for region, url in tqdm(urls.items(), desc="Fetching data"):
    df_temp = fetch_data(url)
    if not df_temp.empty:
        df_temp["region"] = region  # 加入來源區域欄位，便於後續分析
    df_list.append(df_temp)
    time.sleep(5)  # 每次發出請求後暫停 5 秒

# 利用 pd.concat 合併所有 DataFrame（重置索引）
combined_df = pd.concat(df_list, ignore_index=True)

# 顯示結果 (例如：顯示前 5 筆資料)
combined_df


In [ ]:
combined_df.columns

In [ ]:
new_column_names = {
    'AA11': '使用分區',
    'addr': '坐落街道',
    'apply': '申請人',
    'applydate': '申請備查期間',
    'b': '坐落基地',
    'chkdate': '備查完成日期',
    'city': '縣市代號',
    'e': '銷售期間',
    'house': '戶數',
    'id': '編號',
    'idlist': '編號列表',
    'lat': '緯度',
    'ldate': '建照核發日',
    'license': '建照執照',
    'lon': '經度',
    'ma': '主要建材',
    'mark': '起造人',
    'name': '社區名稱',
    'pimg': '圖片路徑',
    'pu': '主要用途',
    'sn': '序號',
    'subid': '子編號',
    'town': '行政區代號',
    'region': '縣市'
}

In [ ]:
df = combined_df.rename(columns=new_column_names, inplace=False)
df

In [ ]:
selected_df = df[["縣市", "行政區代號", "社區名稱", "戶數", "銷售期間", "坐落街道", "主要用途", "使用分區", "備查完成日期", "建照核發日", "建照執照", "經度", "緯度"]]
selected_df

In [ ]:
# 由坐落地址欄位折分出行政區
def parse_admin_region(address):
    # 若不是字串或字串長度為0，直接回傳 None 或空值
    if not isinstance(address, str) or not address:
        return None
    
    # 判斷第二個字是否為「區」
    # 注意：Python 字串的索引從 0 開始
    if len(address) >= 2 and address[1] == "區":
        return address[:2]
    # 判斷第三個字是否為「區」
    elif len(address) >= 3 and address[2] == "區":
        return address[:3]
    # 其餘情況取前三個字
    elif len(address) >= 3:
        return address[:3]
    else:
        # 若字串不足三個字，就直接回傳原字串
        return address

In [ ]:
# 在 df 裡面新增一個「行政區」欄位
selected_df["行政區"] = selected_df["坐落街道"].apply(parse_admin_region)
selected_df.insert(selected_df.columns.get_loc("縣市") + 1, "行政區", selected_df.pop("行政區"))
selected_df

In [ ]:
town_amt = selected_df['行政區'].value_counts()

for town, cnt in town_amt.items():
    print(f"{town}: {cnt}")

In [ ]:
# 定義一個函式來解析銷售期間，回傳 (自售期間, 代銷期間)
def parse_sale_period(s: str):
    # 預設為 None 或空字串
    self_period = None
    agent_period = None

    # 利用正規表達式找出自售期間：匹配 "自售:" 後面所有字串，直到遇到 ";" 或 "代銷:" 或字串結尾
    self_match = re.search(r"自售:(.*?)(?=;|代銷:|$)", s)
    if self_match:
        self_period = self_match.group(1).strip()

    # 利用正規表達式找出代銷期間：匹配 "代銷:" 後面所有字串，直到遇到 ";" 或字串結尾
    agent_match = re.search(r"代銷:(.*?)(?=;|$)", s)
    if agent_match:
        agent_period = agent_match.group(1).strip()

    return self_period, agent_period

In [ ]:
selected_df["自售期間"], selected_df["代銷期間"] = zip(*selected_df["銷售期間"].apply(parse_sale_period))
selected_df

In [ ]:
# 1. 先記錄「銷售期間」欄位的位置
sales_period_idx = selected_df.columns.get_loc("銷售期間")

# 2. 將「自售期間」和「代銷期間」欄位取出 (pop 出來的同時也從 DataFrame 移除)
self_period = selected_df.pop("自售期間")
agent_period = selected_df.pop("代銷期間")

# 3. 利用 insert 方法依序將它們插入到「銷售期間」欄位後面
selected_df.insert(sales_period_idx + 1, "自售期間", self_period)
selected_df.insert(sales_period_idx + 2, "代銷期間", agent_period)

selected_df

In [ ]:
# 定義函式：尋找第一個 7 位數字，若沒有則回傳 None
def find_first_seven_digits(text):
    if not isinstance(text, str):
        return None
    match = re.search(r"\d{7}", text)
    if match:
        return match.group(0)  # 取出第一個符合 7 位數字的字串
    return None

In [ ]:
# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
selected_df["自售起始時間"] = selected_df["自售期間"].apply(find_first_seven_digits)
selected_df["代銷起始時間"] = selected_df["代銷期間"].apply(find_first_seven_digits)

# 將「自售起始時間」欄位移動到「自售期間」欄位後面：
self_time = selected_df.pop("自售起始時間")
self_period_pos = selected_df.columns.get_loc("自售期間")
selected_df.insert(self_period_pos + 1, "自售起始時間", self_time)

# 將「代銷起始時間」欄位移動到「代銷期間」欄位後面：
agent_time = selected_df.pop("代銷起始時間")
agent_period_pos = selected_df.columns.get_loc("代銷期間")
selected_df.insert(agent_period_pos + 1, "代銷起始時間", agent_time)

selected_df


In [ ]:
# 假設 selected_df 已存在，且包含必需欄位：
# 「自售起始時間」、「代銷起始時間」、「銷售期間」、「備查完成日期」、「建照核發日」與「戶數」

# 定義函式：依據規則決定「銷售起始時間」
def compute_sales_start_time(row):
    self_time = row["自售起始時間"]
    agent_time = row["代銷起始時間"]
    
    # Rule 1: 若其中一個有值、另一個為空，則取有值的那一個
    if pd.isna(self_time) and not pd.isna(agent_time):
        return agent_time
    elif pd.isna(agent_time) and not pd.isna(self_time):
        return self_time
    # Rule 2: 如果兩者皆有值，轉為數值比較，取較小者（轉回字串）
    elif not pd.isna(self_time) and not pd.isna(agent_time):
        try:
            self_val = int(self_time)
            agent_val = int(agent_time)
            return str(min(self_val, agent_val))
        except Exception:
            return ""
    # Rule 3: 如果兩者皆無值，則檢查「銷售期間」是否包含"備查"
    elif pd.isna(self_time) and pd.isna(agent_time):
        sales_period = row["銷售期間"]
        if isinstance(sales_period, str) and "備查" in sales_period:
            return row["備查完成日期"]
        else:
            return row["建照核發日"]
    # Rule 4: 其餘情況，回傳空值
    else:
        return ""

# 建立「銷售起始時間」欄位
selected_df["銷售起始時間"] = selected_df.apply(compute_sales_start_time, axis=1)

# 將「銷售起始時間」欄位移至「戶數」欄位之後
sales_start = selected_df.pop("銷售起始時間")
w_index = selected_df.columns.get_loc("戶數")
selected_df.insert(w_index + 1, "銷售起始時間", sales_start)

selected_df

In [ ]:
final_df = selected_df[["縣市", "行政區", "社區名稱", "戶數", "銷售起始時間", "銷售期間",  "自售起始時間", "代銷起始時間", "備查完成日期", "建照核發日", "坐落街道", "主要用途", "使用分區", "建照執照", "經度", "緯度"]]
final_df